### Pricing an American Put Option with QuantLib

In [1]:
import QuantLib as ql 
import matplotlib.pyplot as plt

ql.__version__

'1.31.1'

We are going to price a spread that we have right now which is: 
* Long a put option with strike of 455 on SPY with expiration at 9/1/2023 
* Short a put option with strike of 452.5 on SPY with expiration at 9/1/2023

In [45]:
# option_data
expiration_date = ql.Date(1, 9, 2023)
current_price = 452.39

long_put_strike_price = 455
short_put_strike_price = 452.5

volatility = 0.13 # Using an implied volatility from VIX
dividend_rate = 0.0142
option_type = ql.Option.Put

risk_free_rate = 0.05
day_count = ql.Actual365Fixed()
calendar = ql.UnitedStates(ql.UnitedStates.NYSE)

calculation_date = ql.Date(2, 8, 2023)
ql.Settings.instance().evaluationDate = calculation_date

In [46]:
long_put_payoff = ql.PlainVanillaPayoff(option_type, long_put_strike_price)
short_put_payoff = ql.PlainVanillaPayoff(option_type, short_put_strike_price)
settlement = calculation_date

american_exercise = ql.AmericanExercise(settlement, expiration_date)

long_put_option = ql.VanillaOption(long_put_payoff, american_exercise)
short_put_option = ql.VanillaOption(short_put_payoff, american_exercise)

spot_handle = ql.QuoteHandle(
    ql.SimpleQuote(current_price)
)
flat_rate_term_structure = ql.YieldTermStructureHandle(
    ql.FlatForward(calculation_date, risk_free_rate, day_count)
)

dividend_yield = ql.YieldTermStructureHandle(
    ql.FlatForward(calculation_date, dividend_rate, day_count)
)

flat_vol_ts = ql.BlackVolTermStructureHandle(
    ql.BlackConstantVol(calculation_date, calendar, volatility, day_count)
)

bsm_process = ql.BlackScholesMertonProcess(spot_handle, 
                                           dividend_yield, 
                                           flat_rate_term_structure, 
                                           flat_vol_ts)

In [64]:
steps = 
binomial_engine = ql.BinomialVanillaEngine(bsm_process, 'crr', steps)


In [65]:
long_put_option.setPricingEngine(binomial_engine)

In [66]:
short_put_option.setPricingEngine(binomial_engine)

In [67]:
%%time
long_put_option.NPV()

CPU times: user 1.03 ms, sys: 0 ns, total: 1.03 ms
Wall time: 1.04 ms


6.512367928011533

In [51]:
short_put_option.NPV()

5.284910428210548

In [53]:
%%timeit
long_put_option.NPV() - short_put_option.NPV()

471 ns ± 26.9 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
